In [1]:
# Importing the required libraries
import random
import json
import pickle
import numpy as np

import nltk
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

In [2]:
# Creating a lemmatizer and reading the intents.json file
lemmatizer = WordNetLemmatizer()

intents = json.loads(open('Intents.json').read())

In [3]:
# Creating empty lists

words = []
classes = []
documents = []
ignore_letters = ['?', '!', '.', ',']

In [4]:
# preprocessing the json data
# tokenization

for intent in intents['intents']:
    for pattern in intent['patterns']:
        
        # tokenize each word
        word_list = nltk.word_tokenize(pattern)
        words.extend(word_list)
        
        # add documents in the corpus
        documents.append((word_list, intent['tag']))
        
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:
print(documents)

[(['hi', 'there'], 'greetings'), (['hello'], 'greetings'), (['haroo'], 'greetings'), (['yaw'], 'greetings'), (['wassup'], 'greetings'), (['hi'], 'greetings'), (['hey'], 'greetings'), (['holla'], 'greetings'), (['hello'], 'greetings'), (['bye'], 'goodbye'), (['good', 'bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['Thanks'], 'thanks'), (['okay'], 'thanks'), (['Thank', 'you'], 'thanks'), (['thankyou'], 'thanks'), (['That', "'s", 'helpful'], 'thanks'), (['Awesome', ',', 'thanks'], 'thanks'), (['Thanks', 'for', 'helping', 'me'], 'thanks'), (['wow'], 'thanks'), (['great'], 'thanks'), (['How', 'are', 'you'], 'Ask'), (['How', 'you', 'doing'], 'Ask'), (['How', 'is', 'it', 'going'], 'Ask'), ([], 'noanswer'), (['Who', 'made', 'you'], 'programmer'), (['who', 'designed', 'you'], 'programmer'), (['who', 'programmed', 'you'], 'programmer'), (['what', 'are', 'you', 'doing'], 'activity'), (['You', 'are', 'awesome'], 'appreciate'), (['you', 'are', 'the', 'best'], 'appreciate'), (['you', 'ar

In [6]:
# Lemmatize, lower case each word and remove duplicates

words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_letters]
words = sorted(list(set(words)))
print(words)

["'m", "'s", 'a', 'age', 'ai', 'all', 'allowed', 'am', 'an', 'are', 'artificial', 'aryan', 'assistant', 'awesome', 'be', 'being', 'bend', 'best', 'body', 'bot', 'breathe', 'business', 'bye', 'can', 'chatterbox', 'clone', 'coffee', 'computer', 'contact', 'control', 'cramped', 'creator', 'data', 'date', 'designed', 'developer', 'die', 'do', 'doing', 'entity', 'event', 'favorite', 'favour', 'fight', 'fine', 'for', 'going', 'good', 'great', 'hardware', 'haroo', 'he', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'hobby', 'holla', 'hope', 'how', 'i', 'idea', 'immortal', 'in', 'is', 'it', 'jaw', 'kind', 'know', 'language', 'later', 'laugh', 'lie', 'like', 'linguistic', 'made', 'making', 'malfunction', 'malik', 'mate', 'me', 'motormouth', 'move', 'my', 'name', 'need', 'nice', 'no', 'nope', 'not', 'of', 'ok', 'okay', 'old', 'on', 'operating', 'out', 'over', 'popping', 'product', 'program', 'programmed', 'programmer', 'programming', 'purpose', 'ratchet', 'robot', 'robotics', 'sapient', 's

In [7]:
# sort classes
classes = sorted(list(set(classes)))
print(classes)

['AI', 'Aryan', 'Ask', 'abbr', 'activity', 'age', 'appreciate', 'artificial', 'bend', 'body', 'bot1', 'breathe', 'business', 'chatterbox', 'clone', 'comp', 'computer', 'contact', 'control', 'cramped', 'date', 'death', 'do', 'events', 'exclaim', 'fav', 'fight', 'goodbye', 'greetings', 'greetreply', 'hardware', 'hobby', 'idea', 'imortal', 'lang', 'laugh', 'lie', 'machine', 'malfunction', 'motormouth', 'move', 'name', 'name1', 'need', 'nicetty', 'no', 'noanswer', 'os', 'program', 'programmer', 'programming', 'ratchet', 'robotics', 'robots', 'robotss', 'sapient', 'sense', 'sentiment', 'shoe', 'sound', 'stupid', 'thanks', 'usage', 'who', 'wt']


In [8]:
# creating a pickle file to store the python objects which we will use while predicting

pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

In [9]:
# Now the Machine learning part....

# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    
    # Lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

# shuffle features and converting it into numpy arrays
random.shuffle(training)
training = np.array(training)

# create train and test lists
train_x = list(training[:, 0])
train_y = list(training[:, 1])

print("Training data created")

Training data created


C:\Users\aryan\AppData\Local\Temp\ipykernel_15692\578497123.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [11]:
# Create NN model to predict the responses
model = Sequential()
model.add(Dense(128, input_shape = (len(train_x[0]),), activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation = 'softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(learning_rate = 0.01, decay = 1e-6, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy', optimizer = sgd, metrics = ['accuracy'])

# fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs = 200, batch_size = 5, verbose = 1)
model.save('chatbot.h1', hist) # we will pickle this model to use in the future
print("\n")
print("*"*50)
print("\nModel Created Successfully")

Epoch 1/200
28/28 [==============================] - 0s 2ms/step - loss: 4.2005 - accuracy: 0.0074
Epoch 2/200
28/28 [==============================] - 0s 2ms/step - loss: 4.1253 - accuracy: 0.1103
Epoch 3/200
28/28 [==============================] - 0s 2ms/step - loss: 4.0727 - accuracy: 0.0735
Epoch 4/200
28/28 [==============================] - 0s 2ms/step - loss: 4.0065 - accuracy: 0.0882
Epoch 5/200
28/28 [==============================] - 0s 2ms/step - loss: 3.9708 - accuracy: 0.0809
Epoch 6/200
28/28 [==============================] - 0s 2ms/step - loss: 3.9005 - accuracy: 0.0956
Epoch 7/200
28/28 [==============================] - 0s 2ms/step - loss: 3.8106 - accuracy: 0.1176
Epoch 8/200
28/28 [==============================] - 0s 2ms/step - loss: 3.7840 - accuracy: 0.1250
Epoch 9/200
28/28 [==============================] - 0s 2ms/step - loss: 3.6783 - accuracy: 0.1838
Epoch 10/200
28/28 [==============================] - 0s 2ms/step - loss: 3.5334 - accuracy: 0.1912
Epoch 11/

28/28 [==============================] - 0s 2ms/step - loss: 0.7681 - accuracy: 0.7426
Epoch 84/200
28/28 [==============================] - 0s 2ms/step - loss: 0.7466 - accuracy: 0.7868
Epoch 85/200
28/28 [==============================] - 0s 2ms/step - loss: 0.7685 - accuracy: 0.7721
Epoch 86/200
28/28 [==============================] - 0s 2ms/step - loss: 0.8746 - accuracy: 0.7868
Epoch 87/200
28/28 [==============================] - 0s 2ms/step - loss: 0.7121 - accuracy: 0.7721
Epoch 88/200
28/28 [==============================] - 0s 2ms/step - loss: 0.7032 - accuracy: 0.7647
Epoch 89/200
28/28 [==============================] - 0s 2ms/step - loss: 0.6585 - accuracy: 0.7868
Epoch 90/200
28/28 [==============================] - 0s 2ms/step - loss: 0.7794 - accuracy: 0.7574
Epoch 91/200
28/28 [==============================] - 0s 2ms/step - loss: 0.7211 - accuracy: 0.7574
Epoch 92/200
28/28 [==============================] - 0s 2ms/step - loss: 0.6645 - accuracy: 0.7868
Epoch 93/200


28/28 [==============================] - 0s 3ms/step - loss: 0.4656 - accuracy: 0.8235
Epoch 165/200
28/28 [==============================] - 0s 3ms/step - loss: 0.5142 - accuracy: 0.8309
Epoch 166/200
28/28 [==============================] - 0s 2ms/step - loss: 0.4203 - accuracy: 0.8750
Epoch 167/200
28/28 [==============================] - 0s 2ms/step - loss: 0.4107 - accuracy: 0.8824
Epoch 168/200
28/28 [==============================] - 0s 2ms/step - loss: 0.4126 - accuracy: 0.8529
Epoch 169/200
28/28 [==============================] - 0s 2ms/step - loss: 0.4735 - accuracy: 0.8603
Epoch 170/200
28/28 [==============================] - 0s 2ms/step - loss: 0.2939 - accuracy: 0.8971
Epoch 171/200
28/28 [==============================] - 0s 2ms/step - loss: 0.3212 - accuracy: 0.8971
Epoch 172/200
28/28 [==============================] - 0s 2ms/step - loss: 0.3907 - accuracy: 0.8750
Epoch 173/200
28/28 [==============================] - 0s 2ms/step - loss: 0.3074 - accuracy: 0.8750
Epoc